In [3]:
import ortools
from ortools.constraint_solver import routing_enums_pb2
import ortools.constraint_solver.pywrapcp as pywrapcp
from PIL import Image
from IPython.display import display
import pandas as pd
from itertools import zip_longest
from geopy.distance import geodesic

import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score

import warnings
warnings.filterwarnings("ignore")

import numpy as np

import requests

import folium

import seaborn as sns

import itertools
import math


import branca
import os

C:\Users\MarkosPereira.exc\Anaconda3\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# Load do DataFrame e preprocessamento

In [4]:
# caminho = r"ara_adm (1) - Copia.xlsx".replace("\\","/")
# caminho = r"ara_adm (1).xlsx".replace("\\","/")
caminho = r"demonstração linkedim.xlsx".replace("\\","/")

teste = pd.read_excel(caminho)
teste = teste.drop_duplicates()
teste = teste.reset_index()
teste['lat'] = teste['lat'].astype(str)
teste['long'] = teste['long'].astype(str)
teste['lat'] = teste['lat'].str.replace(",",".")
teste['long'] = teste['long'].str.replace(",",".")
teste['lat'] = teste['lat'].astype(float)
teste['long'] = teste['long'].astype(float)
teste['qtd'] = teste['qtd'].astype(int)

teste

,index,nome_local,qtd,lat,long,rota_original
0,0,Rodoviaria de ponto belo,5,-18.123786,-40.539243,NaN
1,1,Posto de gasolina,3,-18.121471,-40.541137,NaN
2,2,Saida para mucurici,10,-18.117746,-40.539641,NaN
3,3,Ponto em frente a oficina,5,-18.119642,-40.540473,NaN
4,4,Rodoviaria de mucurici,3,-18.096687,-40.521303,NaN
5,5,Ponto em frente ao postinho de saúde,1,-18.094911,-40.518547,NaN
6,6,ponto ao lado do hospital,4,-18.094297,-40.516986,NaN
7,7,Ponto na saida de mucurici,7,-18.096219,-40.515608,NaN
8,8,Rodoviaria de montanha,8,-18.127377,-40.371060,NaN
9,9,Ponto na praça de montanha,9,-18.125985,-40.364109,NaN


# Definição dos parâmetros globais

In [69]:
url = "https://route-and-directions.p.rapidapi.com/v1/routing"


headers = {
    "X-RapidAPI-Key": "38b8f9c50fmsh016cdd649c514d1p1caf6cjsne1d6770fec79",
    "X-RapidAPI-Host": "route-and-directions.p.rapidapi.com"
}



target_index = teste[teste['qtd']==0]['index'].unique()[0]
lat_target_index = teste[teste['qtd']==0]['lat'].unique()[0]
long_target_index = teste[teste['qtd']==0]['long'].unique()[0]
vehicle_capacity = 25
n_routs_desirable = 11


directory = r"C:\Users\MarkosPereira.exc\Downloads\Estudo de roteirização das rotas aracruz\teste demonstração".replace("\\","/")

In [6]:
# target_index
# lat_target_index 
# long_target_index 
# vehicle_capacity = 46
# n_routs_desirable = 16

In [7]:
def verificar_coluna(dataframe, nome_coluna):
    if nome_coluna in dataframe.columns:
        return True
    else:
        return False

# Verificar se a coluna 'start' existe

#### Abaixo a variavel **initialize_starting_points** é a variável vai controlar o fluxo do código, para definir o modo de inicialização das próximas variáveis

In [8]:
if verificar_coluna(teste, 'start') == True:
    teste['start'] = teste['start'].astype(str)
    teste['start'] = teste['start'].str.strip()
    teste['start'] = teste['start'].str.lower()
    initialize_starting_points = False
else:
    starts = None
    n_routs_desirable = n_routs_desirable
    initialize_starting_points = True

# Funções que serão ultilizadas no código

A "distance_matrix" (matriz de distâncias) no contexto de roteamento de veículos, especialmente com a biblioteca ortools da Google, é uma representação matricial das distâncias entre diferentes locais em uma rede viária. Essa matriz é fundamental para resolver problemas de otimização de rotas, ajudando a determinar a distância mais curta ou o tempo de viagem entre todos os pares de locais na rede.

In [9]:
# Função que crie uma distancia comparando a distancie de um ponto em relação aos demais pontos
# O vetores acumulados geram a distance_matrix, que será utilizado no processo de otimização

def create_distance_matrix(DataFrame = pd.DataFrame(), lat=str(), long=str()):
    x = list(zip_longest(DataFrame['lat'][:], DataFrame['long'][:], fillvalue=None))
    distance_matrix = []

    # Calcule as distâncias entre todos os pares de coordenadas
    for coord1 in x:
        distances = []
        for coord2 in x:
            distancia = geodesic(coord1, coord2).kilometers
            distances.append(distancia)
        distance_matrix.append(distances)
        
    return distance_matrix

In [10]:
def create_data_frame(list_list = list(), list_str = list()):
    DataFrame = pd.DataFrame()
    if len(list_list) != len(list_str):
        return 'len(list_list) != len(list_str)'
    else:
        for i in range(len(list_list)):
            DataFrame[list_str[i]] = list_list[i]
        
        return DataFrame

In [11]:
def capturate_start_points(common_column, DataFrame_1 = pd.DataFrame(), DataFrame_2 = pd.DataFrame()):
    if isinstance(common_column, list):
        DataFrame = pd.merge(DataFrame_1, DataFrame_2, on = common_column)
    elif isinstance(objeto, str):
        DataFrame = pd.merge(DataFrame_1, DataFrame_2, on = common_column)
        
    return DataFrame

In [12]:
def cluster_data(data_frame, n_clusters):
    # Selecionar as colunas 'lat' e 'long'
    features = data_frame[['lat', 'long']]

    # Padronizar as features para média zero e desvio padrão 1
    scaler = StandardScaler()
    scaled_features = scaler.fit_transform(features)

    # Executar o algoritmo K-Means com o número de grupos especificado
    kmeans = KMeans(n_clusters=n_clusters, random_state=0)
    data_frame['cluster'] = kmeans.fit_predict(scaled_features)
    data_frame = data_frame.assign(row=1)

    return data_frame

In [13]:
def plot_clusters_on_map(data_frame, lat_col, long_col, cluster_col, colors):
    # Extrair as coordenadas de latitude e longitude e os rótulos de grupo
    lats = data_frame[lat_col]
    longs = data_frame[long_col]
    clusters = data_frame[cluster_col]

    # Criar um mapa Folium
    m = folium.Map(location=[lats.mean(), longs.mean()], zoom_start=5)

    # Adicionar marcadores coloridos por grupo ao mapa
    unique_clusters = clusters.unique()
    for i, cluster in enumerate(unique_clusters):
        cluster_data = data_frame[data_frame[cluster_col] == cluster]
        for _, row in cluster_data.iterrows():
            folium.Marker([row[lat_col], row[long_col]], 
                          icon=folium.Icon(color=colors[i]), 
                          popup=f'Cluster {cluster}').add_to(m)

    # Calcular e adicionar os centroides dos clusters
    centroids = data_frame.groupby(cluster_col)[[lat_col, long_col]].mean().reset_index()
    for _, row in centroids.iterrows():
        folium.Marker([row[lat_col], row[long_col]], 
                      icon=folium.Icon(icon='star', color='black'), 
                      popup=f'Centroide do Cluster {row[cluster_col]}').add_to(m)

    return m, centroids

In [14]:
def haversine_distance(lat1, lon1, lat2, lon2):
    # Raio da Terra em quilômetros
    radius = 6371

    # Converte graus para radianos
    lat1 = np.radians(lat1)
    lon1 = np.radians(lon1)
    lat2 = np.radians(lat2)
    lon2 = np.radians(lon2)

    # Diferenças de latitude e longitude
    dlat = lat2 - lat1
    dlon = lon2 - lon1

    # Fórmula de Haversine
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    distance = radius * c

    return distance

In [15]:
def calcular_proporcionais(valores, valor_base):
    # Verifica se a lista de valores não está vazia
    if not valores:
        return []

    # Calcula a soma dos valores na lista
    soma_valores = sum(valores)

    # Calcula os fatores de escala para manter a proporção
    fatores_escala = [valor_base / soma_valores * valor for valor in valores]

    # Arredonda os fatores de escala para números inteiros
    fatores_inteiros = [round(fator) for fator in fatores_escala[::1]]

    # Ajusta para garantir que a soma seja igual ao valor base
    diferenca = valor_base - sum(fatores_inteiros)
    fatores_inteiros[-1] += diferenca

    return fatores_inteiros

In [16]:
def calculate_best_starting_points_with_destination(latitudes, longitudes, demands, destination_latitude, destination_longitude, num_vehicles):
    # Índices possíveis para pontos de início
    possible_starts = list(range(len(latitudes)))

    # Combinar os índices dos pontos de início com a demanda total
    start_combinations = itertools.combinations(possible_starts, num_vehicles)

    best_starts_indices = min(start_combinations, key=lambda indices: sum(demands[i] for i in indices))

    # Obter as coordenadas correspondentes aos melhores pontos de início
    best_starting_points = [(latitudes[i], longitudes[i]) for i in best_starts_indices]

    # Adicionar o ponto de destino
    best_starting_points.append((destination_latitude, destination_longitude))

    return best_starting_points

In [17]:
def capturate_start_points(list_to_start_points = list()):
    start_latitudes = list()
    start_longitudes = list()
    for points in list_to_stars_points:
        start_latitudes.append(point[0])
        start_longitudes.append(point[1])
    return start_latitudes, start_longitudes

In [18]:
def plot_clusters_on_map_multicolor(lista_lat, lista_lon, chave_lat, chave_lon):
    # Criar conjunto de coordenadas chave para verificação
    coordenadas_chave = set(zip(chave_lat, chave_lon))

    # Filtrar coordenadas não chave que não estão nas coordenadas chave
    coordenadas_nao_chave_filtradas = [
        (lat, lon) for lat, lon in zip(lista_lat, lista_lon) if (lat, lon) not in coordenadas_chave
    ]

    # Separar coordenadas chave que não estão presentes nas outras listas
    coordenadas_chave_filtradas = [
        (lat, lon) for lat, lon in zip(chave_lat, chave_lon)
        if (lat, lon) not in set(zip(lista_lat, lista_lon))
    ]

    # Configurar o estilo do gráfico de dispersão
    sns.set(style='whitegrid')
    plt.figure(figsize=(10, 8))

    # Plotar os pontos não chave em escala de cinza
    plt.scatter(
        [lon for _, lon in coordenadas_nao_chave_filtradas],
        [lat for lat, _ in coordenadas_nao_chave_filtradas],
        color='lightgrey',
        label='Pontos não chave',
        s=100
    )

    # Plotar os pontos chave filtrados em vermelho
    plt.scatter(
        [lon for _, lon in coordenadas_chave_filtradas],
        [lat for lat, _ in coordenadas_chave_filtradas],
        color='red',
        label='Pontos chave',
        s=150,
        marker='X'
    )

    # Configurar rótulos e título
    plt.xlabel('Longitude')
    plt.ylabel('Latitude')
    plt.title('Pontos no Mapa')

    # Mostrar a legenda
    plt.legend()

    # Exibir o mapa
    plt.show()

In [19]:
import matplotlib.pyplot as plt

def plot_points_on_map(lista_lat, lista_lon):
    # Configurar o estilo do gráfico de dispersão
    plt.figure(figsize=(10, 8))

    # Plotar os pontos em escala de cinza
    plt.scatter(lista_lon, lista_lat, color='lightgrey', label='Pontos', s=100)

    # Configurar rótulos e título
    plt.xlabel('Longitude')
    plt.ylabel('Latitude')
    plt.title('Pontos no Mapa')

    # Mostrar a legenda
    plt.legend()

    # Exibir o mapa
    plt.show()

In [20]:
def create_data_model(distance_matrix, demands, vehicle_capacities, n_routs_desirable, depot, starts, ends):
    """Stores the data for the problem."""
    data = {}
    data["distance_matrix"] = distance_matrix
    data["demands"] = demands
    data["vehicle_capacities"] = vehicle_capacities
    data["num_vehicles"] = n_routs_desirable
    data["depot"] = depot
    data["starts"] = starts
    data["ends"] = ends
    return data


def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    print(f"Objective: {solution.ObjectiveValue()}")
    total_distance = 0
    total_load = 0
    max_route_distance = 0
    #####
    df_from_consult = list()
    #####
    for vehicle_id in range(data["num_vehicles"]):
        index = routing.Start(vehicle_id)
        plan_output = f"Route for vehicle {vehicle_id}:\n"
        route_distance = 0
        route_load = 0
        #########
        plans = list()
        points = list()
        ocupacao = list()
        ##########
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route_load += data["demands"][node_index]
            plan_output += f" {node_index} Load({route_load}) -> "
            points.append(int(node_index))
            ocupacao.append(int(route_load))
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id
            )
        #####################
#         print(manager.IndexToNode(index))
#         print("!!!!!!!!!!!!!")
        #######################

        plan_output += f" {manager.IndexToNode(index)} Load({route_load})\n"
        ##########
        auxiliar_df = pd.DataFrame({'index': points, 'qtd': ocupacao})
        line_final = pd.DataFrame({'index': [manager.IndexToNode(index)], 'qtd': [0]})
        df_from_consult.append(pd.concat([auxiliar_df, line_final]))
        ##########
        plan_output += f"Distance of the route: {route_distance}m\n"
        plan_output += f"Load of the route: {route_load}\n"
        plans.append(plan_output)

#         print(plan_output)
        total_distance += route_distance
        total_load += route_load
#     print(f"Total distance of all routes: {total_distance}m")
#     print(f"Total load of all routes: {total_load}")

    return df_from_consult



def main():
    """Solve the CVRP problem."""
    # Instantiate the data problem.
    data = create_data_model(distance_matrix, demands, vehicle_capacities, n_routs_desirable, depot, starts, ends)

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(
        len(data["distance_matrix"]), data["num_vehicles"], data["starts"], data["ends"]
    )

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)

    # Create and register a transit callback.
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data["distance_matrix"][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Add Capacity constraint.
    def demand_callback(from_index):
        """Returns the demand of the node."""
        # Convert from routing variable Index to demands NodeIndex.
        from_node = manager.IndexToNode(from_index)
        return data["demands"][from_node]

    demand_callback_index = routing.RegisterUnaryTransitCallback(demand_callback)
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # null capacity slack
        data["vehicle_capacities"],  # vehicle maximum capacities
        True,  # start cumul to zero
        "Capacity",
    )

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.AUTOMATIC
#         routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
    )
    search_parameters.local_search_metaheuristic = (
        routing_enums_pb2.LocalSearchMetaheuristic.AUTOMATIC
    )
    search_parameters.log_search = True
    search_parameters.time_limit.FromSeconds(15)
    
    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)
    
    # Print solution on console.
    if solution:
        x = print_solution(data, manager, routing, solution)
        return x

# x = main()

In [21]:
def capturate_points_in_routs(starts_and_end_points):
    latitudes = list()
    longitudes = list()
    
    #
    distance_interpoints = list()
    distance_interpoints.append(0)
    
    time_interpoints = list()
    time_interpoints.append(0)
    
    for start_and_end_points in starts_and_end_points:
#         print(start_and_end_points)
        querystring = {"waypoints":f"{start_and_end_points}","mode":"drive"}
        response = requests.get(url, headers=headers, params=querystring, verify=False)
        
        for values in response.json()['features'][0]['geometry']['coordinates'][0]:
            latitudes.append(values[1])
            longitudes.append(values[0])
        #################
        
        time_interpoints.append(response.json()['features'][0]['properties']['time'])
        distance_interpoints.append(response.json()['features'][0]['properties']['distance'])
         
            
    return latitudes, longitudes, distance_interpoints, time_interpoints

In [22]:
def capture_move(data_frame = pd.DataFrame()):
    moves = list()
    for index in data_frame.index[1:]:
        move = str()
        lat0 = str(data_frame.iloc[index]['lat1'])
        long0 = str(data_frame.iloc[index]['long1'])
        lat1 = str(data_frame.iloc[index]['lat'])
        long1 = str(data_frame.iloc[index]['long'])
        move = lat0+','+long0+'|'+lat1+','+long1
        moves.append(move)
    return moves

In [23]:
def plotar_mapa(latitudes, longitudes, latitudes_chaves, longitudes_chaves, dicas_ferramentas, name, directory):
    # Cria um objeto de mapa Folium
    mapa = folium.Map(location=[sum(latitudes) / len(latitudes), sum(longitudes) / len(longitudes)], zoom_start=10)

    # Adiciona marcadores para cada par de latitude e longitude (invisíveis)
    for lat, lon in zip(latitudes, longitudes):
        folium.Marker(location=[lat, lon], opacity=0).add_to(mapa)

    # Adiciona uma linha conectando os pontos de latitude e longitude (duas vezes mais grossa)
    folium.PolyLine(zip(latitudes, longitudes), color='blue', weight=5, opacity=1).add_to(mapa)

    # Adiciona marcadores para os pontos-chave com dicas de ferramentas
    for lat, lon, dica in zip(latitudes_chaves, longitudes_chaves, dicas_ferramentas):
        folium.Marker(location=[lat, lon], popup=dica, icon=folium.Icon(color='red')).add_to(mapa)

    # Salva o mapa em um arquivo HTML
    mapa.save(f'{directory}/{name}.html')

    return mapa

In [24]:
# def haversine_distance(lat1, lon1, lat2, lon2):
def distance_rout(latitudes, longitudes):
    auxilair_df = pd.DataFrame({'lat': latitudes, 'long': longitudes})
    auxilair_df['lat1'] = auxilair_df['lat'].shift(1)
    auxilair_df['long1'] = auxilair_df['long'].shift(1)
    auxilair_df['distance'] = haversine_distance(auxilair_df['lat'], auxilair_df['lat1'], auxilair_df['long'], auxilair_df['long1'])
    distance = auxilair_df['distance'].sum()
    return distance

In [25]:
def list_to_tuple(list1 = list(), list2 = list()):
    list_tuples= list()
    if len(list1) != len(list2):
        print("len(list1) != len(list2)")
        return 0
    
    for i in range(len(list1)):
        tuple_points = (list1[i], list2[i])
        list_tuples.append(tuple_points)
        
    return list_tuples

In [26]:
def create_summary_lines(locations, colors, markers, tooltips):
    kw = {"opacity": 1.0, "weight": 4}
    lat = lon = 0  # Corrigido de lat = long = 0 para lat = lon = 0
    zoom_start = 2  # Defina seu próprio valor de zoom_start
    
    m = folium.Map(location=[lat, lon], zoom_start=zoom_start)
    
    for i in range(len(locations)):
        # Adiciona a polilinha ao mapa
        folium.PolyLine(
            locations=locations[i],
            line_cap="round",
            color=colors[i],  # Adiciona a cor específica para a linha
            **kw,
        ).add_to(m)
        
        if i < len(markers):  # Verifica se há marcadores disponíveis para este trajeto
            for j in range(len(markers[i])):
                # Adiciona os marcadores com seus tooltips correspondentes
                folium.Marker(
                    location=markers[i][j],
                    icon=folium.Icon(color=colors[i]),
                    popup=tooltips[i][j]
                ).add_to(m)
        
    return m

In [27]:
def creat_legend(colors = list(), start=int()):
    string = """
    {% macro html(this,kwargs) %}

    <div style = "position: fixed;
    bottom: 12px;
    left: 600px;
    width: 1200px;
    height: 150px;
    font-size: 14px;
    z-index: 9999;
    ">
    <p><a style = "color: black; margin-left: 20px;"></a><b>Legenda</b></p>"""
    
    pos = start
    

    for color in colors:   
        string = string + f"""<p><a style = "color:{color}; margin-left: 20px;">&FilledSmallSquare;
        </a>rout_{pos+1}</p>"""
        pos+=1
        

    string = string + """</div>"""

    string = string + """<div style = "position: fixed;
    bottom: 12px;
    left: 600px;
    width: 1200px;
    height: 150px;
    font-size: 14px;
    background-color: white;
    z-index: 9998;
    opacity: 0.7;
    border: 2px solid grey;
    border-radius: 6px;
    ">
    </div>

    {% endmacro %}"""

    return string 

In [28]:
def formate_time(segundos):
    horas = segundos // 3600
    minutos = (segundos % 3600) // 60
    segundos = segundos % 60

    tempo_formatado = "{:02d}:{:02d}:{:02d}".format(int(horas), int(minutos), int(segundos))
    return tempo_formatado

In [29]:
def create_new_colors_vector(original_vector, n):
    new_vector = []

    # Determina quantas vezes o vetor original cabe em n
    rep = n // len(original_vector)
    res = n % len(original_vector)

    # Adiciona o vetor original repetidas vezes no novo vetor
    for _ in range(rep):
        new_vector.extend(original_vector)

    # Adiciona o restante dos elementos para preencher até alcançar o tamanho n
    new_vector.extend(original_vector[:res])

    return new_vector

# veror de cores tolerados em folium.Icon(), são apenas essas as opções disponiveis
vetor_original = ['red', 'blue', 'green', 'purple', 'orange', 'darkred', 'grey',
                  'beige', 'darkblue', 'darkgreen', 'cadetblue',
                  'pink', 'blue', 'green']

In [30]:
def processing_df_from_locations(df_from_locations = pd.DataFrame(), n_acentos=int(), pos = int()):
    auxiliar_df = pd.DataFrame()
    auxiliar_df['N_stops'] = [len(df_from_locations)-1]
    auxiliar_df['star_point'] = [df_from_locations['nome_local'][0]]
    auxiliar_df['max_capacity'] = [n_acentos]
    auxiliar_df['occupation'] = [df_from_locations['qtd_y'].sum()]
    auxiliar_df['occupancy_rate'] = [float(auxiliar_df['occupation']/auxiliar_df['max_capacity'])]
    auxiliar_df['rout_distance'] = [df_from_locations['distance_interpoints'].sum()/1000]
    auxiliar_df['time_estation'] = [df_from_locations['time_interpoints'].sum()/60]
    auxiliar_df['time_estation_formatted'] = [formate_time(df_from_locations['time_interpoints'].sum())]
    auxiliar_df = auxiliar_df.assign(rout = f"rout_{pos+1}")
    
    return auxiliar_df

In [31]:
def plot_points_with_tooltips(latitudes, longitudes, latitudes_chave, longitudes_chave, tooltips, tooltips_chave):
    # Criando o mapa
    m = folium.Map(location=[sum(latitudes)/len(latitudes), sum(longitudes)/len(longitudes)], zoom_start=5,
                  name="Pontos de inicio das rotas (definidos manualmente)")

    # Adicionando os pontos normais em escala de cinza com tooltips
    for lat, lon, tooltip in zip(latitudes, longitudes, tooltips):
        folium.Marker(
            location=[lat, lon],
            icon=folium.Icon(color='lightgray'),
            popup=tooltip
        ).add_to(m)

    # Adicionando os pontos-chave em vermelho com tooltips
    for lat, lon, tooltip in zip(latitudes_chave, longitudes_chave, tooltips_chave):
        folium.Marker(
            location=[lat, lon],
            icon=folium.Icon(color='red'),
            popup=tooltip
        ).add_to(m)

    return m

In [32]:
def visualize_available_points(latitudes, longitudes, zoom_level, title, tooltips=None):
    m = folium.Map(location=[sum(latitudes)/len(latitudes), sum(longitudes)/len(longitudes)], zoom_start=zoom_level, name=title)

    # Verificar se tooltips foram fornecidos
    if tooltips is not None and len(tooltips) == len(latitudes):
        # Adicionando marcadores aos pontos com tooltips
        for lat, lon, tooltip in zip(latitudes, longitudes, tooltips):
            folium.Marker(
                location=[lat, lon],
                icon=folium.Icon(color='gray'),
                popup=tooltip
            ).add_to(m)
    else:
        # Adicionando marcadores aos pontos sem tooltips
        for lat, lon in zip(latitudes, longitudes):
            folium.Marker(
                location=[lat, lon],
                icon=folium.Icon(color='gray'),
            ).add_to(m)

    return m

In [33]:
def plot_clusters_on_map_not_html(data_frame, lat_col, long_col, cluster_col, colors=list(), directory="", name=""):
    # Extrair as coordenadas de latitude e longitude e os rótulos de grupo
    lats = data_frame[lat_col]
    longs = data_frame[long_col]
    clusters = data_frame[cluster_col]

    # Configurar o estilo do gráfico de dispersão
    sns.set(style='whitegrid')
    plt.figure(figsize=(10, 8))

    # Plotar os pontos coloridos por grupo
    sns.scatterplot(x=longs, y=lats, hue=clusters, palette=colors, s=100, legend='full')

    # Configurar rótulos e título
    plt.xlabel('Longitude')
    plt.ylabel('Latitude')
    plt.title('Grupos de Dados no Mapa')

    # Mostrar a legenda
    plt.legend(title='Grupos', loc='upper right')

    # Calcular e plotar os centroides dos clusters
    centroids = data_frame.groupby(cluster_col)[[lat_col, long_col]].mean().reset_index()
#     sns.scatterplot(x=centroids[long_col], y=centroids[lat_col], hue=centroids[cluster_col],
#                     palette=colors, s=200, marker='X', legend=False, ax=plt.gca())

    # Salvar a figura no diretório fornecido
    if directory and os.path.isdir(directory):
        plt.savefig(os.path.join(directory, f'{name}.jpeg'))
        plt.close()
    else:
        plt.show()

    return centroids

In [55]:
def obter_localizacoes(latitudes, longitudes):
    locais = []
    cont = 0
    for lat, lon in zip(latitudes, longitudes):
        url = f"https://nominatim.openstreetmap.org/reverse?lat={lat}&lon={lon}&format=json"
        response = requests.get(url, verify=False)
        data = response.json()
    
            
        if 'town' in data['address'] and 'city_district' in data['address']:
            local = data['address']['town']+','+data['address']['city_district']
            locais.append(local)
        
        elif 'town' in data['address']:
            local = data['address']['town']
            locais.append(local)
        
        elif 'city' in data['address']:
            locais.append(data['address']['city'])
        
        elif 'suburb' in data['address']:
            locais.append(data['address']['suburb'])
        
        elif 'city_district' in data['address']:
            locais.append(data['address']['city_district'])
            
        else:
            locais.append(cont)
            
        cont+=1
    return locais

# 1º Ramificação

## Caso initialize_starting_points seja False

Como a coluna 'start' existe no data frame, os potenciais pontos de partida das linhas ja foram definidos pelo usario

In [35]:
if initialize_starting_points == False:
    # Identificação dos pontos sinalizados pelo usar
    points_to_starts = list(teste[teste['start']=='x']['index'])
    starts = points_to_starts
    # definição dos numero de veículos na frota
    # Definição dos pontos de latitude e longitudes onde as rotas devem iniciar
    # Essa é uma especificação do usuário deve ser respeitada
    # essas latitudes e longitudes serão plotadas no mapara para que o osuario possa ver
    inicial_latitudes = list(teste[teste['index'].isin(starts)]['lat'])
    inicial_longitudes = list(teste[teste['index'].isin(starts)]['long'])

In [36]:
if initialize_starting_points == False:
# Visualização e confirmação dos pontos de partida pelo usuário
    mapa = plot_points_with_tooltips(list(teste['lat']), list(teste['long']),
                                 list(teste[teste['index'].isin(starts)]['lat']),
                                 list(teste[teste['index'].isin(starts)]['long']),
                                 list(teste['nome_local']), list(teste[teste['index'].isin(starts)]['nome_local']))
    mapa.save(f"{directory}/Pontos de start das rotas.html")  # Salva o mapa em um arquivo HTML para visualização

In [37]:
if initialize_starting_points == False:
    distance_matrix = create_distance_matrix(teste, 'lat', 'long')
    demands = list(teste['qtd'])
    depot = int(target_index)
    starts = points_to_starts
    ends = [int(target_index) for _ in range(n_routs_desirable)]
    vehicle_capacities = [vehicle_capacity for _ in range(n_routs_desirable)]

In [38]:
if initialize_starting_points == False:
    n_clusters = int(n_routs_desirable**0.5)+2
    # Chamar a função para agrupar os dados em n grupos (de acoroo com o numero de rotas)
    lista_locais = obter_localizacoes(list(teste['lat']), list(teste['long']))
    teste['cluster'] = lista_locais
    clustered_df = teste
    clustered_df = clustered_df.assign(row=1)
    # Chamar a função para plotar os grupos no mapa
    colors = create_new_colors_vector(vetor_original, n_clusters)
    x = plot_clusters_on_map(clustered_df, 'lat', 'long', 'cluster', colors)
    y = clustered_df[['cluster', 'qtd', 'row']].groupby('cluster').sum(['qtd', 'row']).sort_values('qtd', ascending=False)
    y = y.reset_index()
    z = pd.merge(x[1], y, on='cluster')
    z= z.assign(lat1= lat_target_index) 
    z= z.assign(long1= long_target_index)
    z['distance'] = haversine_distance(z['lat'], z['long'], z['lat1'], z['long1'])
    z = z.sort_values('qtd', ascending=False)
    z['pesos'] = z['distance']*z['qtd']
    base = n_routs_desirable
    resultado = calcular_proporcionais(list(z['pesos']), base)
    z['n_routs'] = resultado
    n_routs_desirable = len(starts)

# 2 º Ramificação

## Caso initialize_starting_points seja True

Como a coluna 'start' não existe no data frame, os potenciais pontos de partida das linhas devem ser definidos matematicamente

In [56]:
list_to_stars_points = list()
if initialize_starting_points == True:
    n_clusters = int(n_routs_desirable**0.5)+2
    # Chamar a função para agrupar os dados em n grupos (de acoroo com o numero de rotas)
    lista_locais = obter_localizacoes(list(teste['lat']), list(teste['long']))
    teste['cluster'] = lista_locais
    clustered_df = teste
    clustered_df = clustered_df.assign(row=1)
    # Chamar a função para plotar os grupos no mapa
    colors = create_new_colors_vector(vetor_original, len(clustered_df['cluster'].unique()))
    x = plot_clusters_on_map(clustered_df, 'lat', 'long', 'cluster', colors)
    y = clustered_df[['cluster', 'qtd', 'row']].groupby('cluster').sum(['qtd', 'row']).sort_values('qtd', ascending=False)
    y = y.reset_index()
    z = pd.merge(x[1], y, on='cluster')
    z= z.assign(lat1= lat_target_index) 
    z= z.assign(long1= long_target_index)
    z['distance'] = haversine_distance(z['lat'], z['long'], z['lat1'], z['long1'])
    z = z.sort_values('qtd', ascending=False)
    z['pesos'] = z['distance']*z['qtd']
    base = n_routs_desirable
    resultado = calcular_proporcionais(list(z['pesos']), base)
    z['n_routs'] = resultado

In [57]:
if initialize_starting_points == True:
    points_to_starts = list()
    for cluster in z['cluster'].unique():
        new_df = clustered_df[clustered_df['cluster']==cluster]
        centroide_to_destino = z[z['cluster']==cluster]['distance'].unique()[0]
        lat_centroide = z[z['cluster']==cluster]['lat'].unique()[0]
        long_centroide = z[z['cluster']==cluster]['long'].unique()[0]
        n_points_to_star = z[z['cluster']==cluster]['n_routs'].unique()[0]

        distances = list()
        for index in new_df['index']:
            lat = new_df[new_df['index']==index]['lat'].unique()[0]
            long = new_df[new_df['index']==index]['long'].unique()[0]
            # calcula a distancia do ponto ao centroide
            distance_point_to_centroide = haversine_distance(lat_centroide, long_centroide, lat, long)
            distance_point_to_destine = haversine_distance(lat_target_index, long_target_index, lat, long)
            distances.append(distance_point_to_centroide+distance_point_to_destine)


        new_df['distances'] = distances
        points_starts = list(new_df.sort_values('distances', ascending=False)['index'][0:n_points_to_star])
        points_to_starts.extend(points_starts)

In [58]:
if initialize_starting_points == True:
# Visualização e confirmação dos pontos de partida pelo usuário
    mapa = plot_points_with_tooltips(list(teste['lat']), list(teste['long']),
                                 list(teste[teste['index'].isin(points_to_starts)]['lat']),
                                 list(teste[teste['index'].isin(points_to_starts)]['long']),
                                 list(teste['nome_local']), list(teste[teste['index'].isin(points_to_starts)]['nome_local']))
    mapa.save(f"{directory}/Pontos de start das rotas.html")  # Salva o mapa em um arquivo HTML para visualização

# Visualizações importantes

## Distribuição dos pontos de embarque disponíveis

In [59]:
mapa = visualize_available_points(list(teste['lat']), list(teste['long']), 5, 'pontos disponiveis', list(teste['nome_local']))
mapa.save(f'{directory}/pontos_disponivel.html')

In [60]:
x[0].save(f'{directory}/grupos_de_pontos_disponiveis.html')

In [61]:
new_x = plot_clusters_on_map_not_html(clustered_df, 'lat', 'long', 'cluster', colors, directory, "grupos_de_pontos_disponiveis")

In [62]:
z.to_excel(f"{directory}/demanda_por_grupo_de_pontos.xlsx", index=False)

# Definição dos parametros finais

In [63]:
distance_matrix = create_distance_matrix(teste, 'lat', 'long')
demands = list(teste['qtd'])
depot = int(target_index)
starts = points_to_starts
ends = [int(target_index) for _ in range(n_routs_desirable)]
vehicle_capacities = [vehicle_capacity for _ in range(n_routs_desirable)]

In [64]:
print(len(distance_matrix))
print(len(demands))
print(depot)
print(len(starts))
print(starts)
print(len(ends))
print(ends)
print(len(vehicle_capacities))
print(vehicle_capacities)

46
46
45
11
[33, 28, 8, 9, 23, 2, 3, 14, 35, 40, 4]
11
[45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45]
11
[25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25]


In [65]:
print(type(distance_matrix))
print(type(demands))
print(type(depot))
print(type(starts[0]))
print(starts)
print(type(ends))
print(ends)
print(type(vehicle_capacities[0]))
print(vehicle_capacities)

<class 'list'>
<class 'list'>
<class 'int'>
<class 'int'>
[33, 28, 8, 9, 23, 2, 3, 14, 35, 40, 4]
<class 'list'>
[45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45]
<class 'int'>
[25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25]


# Criação das rotas e resumos unitários

In [66]:
new_routs = main()

Objective: 738


In [70]:
dfs_from_locations = list()
sumary_latitudes = list()
sumary_longitudes = list()
suamry_markers_latitudes = list()
suamry_markers_longitudes = list()
sumary_tooltips = list()
from_final_resume = list()

for value in new_routs:
    pos = 1
    for y in new_routs:
        df_from_locations = y.merge(teste, on = ['index'], how='inner')
        df_from_locations['lat1'] = df_from_locations['lat'].shift(1)
        df_from_locations['long1'] = df_from_locations['long'].shift(1)
        moves = capture_move(df_from_locations)
        latitudes, longitudes, distance_interpoints, time_interpoints = capturate_points_in_routs(moves)
        df_from_locations['distance_interpoints'] = distance_interpoints
        df_from_locations['time_interpoints'] = time_interpoints
        from_final_resume.append(df_from_locations)
        df_from_locations.to_excel(f"{directory}/rout_{pos}.xlsx", index=False)
        distance_rout(latitudes, longitudes)
        plotar_mapa(latitudes, longitudes, list(df_from_locations['lat']),  list(df_from_locations['long']), df_from_locations['nome_local'], f"rout_{pos}", directory)
        
        sumary_latitudes.append(latitudes)
        sumary_longitudes.append(longitudes)
        suamry_markers_latitudes.append(list(df_from_locations['lat']))
        suamry_markers_longitudes.append(list(df_from_locations['long']))
        sumary_tooltips.append(list(df_from_locations['nome_local']))
        
        pos += 1
        if pos > n_routs_desirable:
            break
    if pos > n_routs_desirable:
        break

# Resumo final (TABULAR)

In [71]:
final_resumes = list()
pos = 0
for df_from_locations in from_final_resume:
    final_resume = processing_df_from_locations(df_from_locations, vehicle_capacity, pos)
    final_resumes.append(final_resume)
    pos+=1
final_resume = pd.concat(final_resumes)
final_resume.to_excel(f"{directory}/final_resume.xlsx", index=False)

# Resumos finais (Gráficos)

In [72]:
# Chamada da função para criar o novo vetor de tamanho n
colors = create_new_colors_vector(vetor_original, n_routs_desirable)

# print(len(colors))
locations = list()
for sl in range(len(sumary_latitudes)):
    locations.append(list_to_tuple(sumary_latitudes[sl], sumary_longitudes[sl]))

markers = list()
for sl in range(len(suamry_markers_latitudes)):
    markers.append(list_to_tuple(suamry_markers_latitudes[sl], suamry_markers_longitudes[sl]))



start_saves = 0
end_saves = 3
for n in range(int(len(locations)/3)+1): 
    m = create_summary_lines(locations[start_saves:end_saves], colors[start_saves:end_saves], markers[start_saves:end_saves], sumary_tooltips[start_saves:end_saves])
    legenda_mapa = creat_legend(colors[start_saves:end_saves], start_saves)
    legenda = branca.element.MacroElement()
    legenda._template = branca.element.Template(legenda_mapa)
    m.add_child(legenda)
    m.save(f"{directory}/resume_graphical_routs_{start_saves+1}_at_rout_{end_saves}.html")
    start_saves = end_saves
    end_saves = end_saves+3

In [73]:
colors = create_new_colors_vector(vetor_original, n_routs_desirable)
m = create_summary_lines(locations, colors, markers, sumary_tooltips)
m.save(f"{directory}/resume_graphical_routs.html")

In [74]:
url = f"https://nominatim.openstreetmap.org/reverse?lat={teste[teste['index']==8]['lat'].unique()[0]}&lon={teste[teste['index']==8]['long'].unique()[0]}&format=json"
response = requests.get(url, verify=False)
data = response.json()
data

{'place_id': 9257944,
 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. http://osm.org/copyright',
 'osm_type': 'way',
 'osm_id': 180958295,
 'lat': '-18.127393091022455',
 'lon': '-40.37106005813069',
 'class': 'highway',
 'type': 'residential',
 'place_rank': 26,
 'importance': 0.10000999999999993,
 'addresstype': 'road',
 'name': 'Avenida Antônio Paulino',
 'display_name': 'Avenida Antônio Paulino, Centro, Montanha, Região Geográfica Imediata de São Mateus, Região Geográfica Intermediária de São Mateus, Espírito Santo, Região Sudeste, 29890-000, Brasil',
 'address': {'road': 'Avenida Antônio Paulino',
  'suburb': 'Centro',
  'city_district': 'Montanha',
  'town': 'Montanha',
  'municipality': 'Região Geográfica Imediata de São Mateus',
  'state_district': 'Região Geográfica Intermediária de São Mateus',
  'state': 'Espírito Santo',
  'ISO3166-2-lvl4': 'BR-ES',
  'region': 'Região Sudeste',
  'postcode': '29890-000',
  'country': 'Brasil',
  'country_code': 'br'},
 'boundingb

In [ ]:
# new_dict = {'place_id': 9298029, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. http://osm.org/copyright', 'osm_type': 'way', 'osm_id': 939530130, 'lat': '-19.83939205', 'lon': '-40.0810956', 'class': 'man_made', 'type': 'cooling', 'place_rank': 30, 'importance': 9.99999999995449e-06, 'addresstype': 'man_made', 'name': '', 'display_name': 'ES-010, Barra do Riacho, Aracruz, Região Geográfica Imediata de Linhares, Região Geográfica Intermediária de São Mateus, Espírito Santo, Região Sudeste, 29197-539, Brasil', 'address': {'road': 'ES-010', 'suburb': 'Barra do Riacho', 'city_district': 'Aracruz', 'town': 'Aracruz', 'municipality': 'Região Geográfica Imediata de Linhares', 'state_district': 'Região Geográfica Intermediária de São Mateus', 'state': 'Espírito Santo', 'ISO3166-2-lvl4': 'BR-ES', 'region': 'Região Sudeste', 'postcode': '29197-539', 'country': 'Brasil', 'country_code': 'br'}, 'boundingbox': ['-19.8394705', '-19.8393136', '-40.0813088', '-40.0808824']}

In [ ]:
# new_dict.keys()

In [ ]:
# new_dict['address']['suburb']

In [ ]:
# lista_locais = obter_localizacoes(list(teste['lat'][5:6]), list(teste['long'][5:6]))

In [ ]:
# lista_locais